In [2]:
# Library imports
import pyforest
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from pprint import pprint
from time import sleep
import time
import seaborn as sns

from turtle import forward
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.utils.data import DataLoader, TensorDataset

#### **Hyperparams and data-loading**

In [3]:
train_edges = np.load('movielens/train_edges.npy')
user_list = train_edges[:, 0]
item_list = train_edges[:, 1]
rating_list = train_edges[:, 2].astype('float32')

n_users = user_list.max() + 1 
n_items = item_list.max() + 1
n_samples = len(rating_list)

n_users, n_items, n_samples

(943, 1682, 200000)

#### **Neural collaborative filtering**

In [33]:
class NCF(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(NCF, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)
        self.fc = nn.Linear(n_factors * 2, 1)
        # self.user_emb.weight.data.uniform_(0, 0.5)
        # self.item_emb.weight.data.uniform_(0, 0.5)
        # self.fc.weight.data.uniform_(0, 0.5)

    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        features = torch.concat([u, i], dim = 1)
        x = self.fc(features)
        out = torch.sigmoid(x)
        return out

In [8]:
class NCF(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(NCF, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)
        self.fc1 = nn.Linear(n_factors * 2, n_factors)
        # self.fc2 = nn.Linear(n_factors, n_factors // 2)
        self.fc2 = nn.Linear(n_factors, 1)
        # self.fc3 = nn.Linear(n_factors // 2, 1)

    def forward(self, user, item):
        tanh = nn.Tanh()
        sigmoid = nn.Sigmoid()

        u = self.user_emb(user)
        i = self.item_emb(item)
        features = torch.concat([u, i], dim = 1)
        x = self.fc1(features)
        x = tanh(x)
        x = self.fc2(x)
        # x = tanh(x)
        # x = self.fc3(x)
        x = sigmoid(x)
        return x

In [5]:
class CollaborativeFiltering(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(CollaborativeFiltering, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)

    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        dot = (u * i).sum(1)
        return torch.sigmoid(dot)

In [6]:
def get_accuracy(y_hat, y):
    y = y.clone().int()
    y_hat = (y_hat.clone() > 0.5).int()
    accuracy = (y == y_hat).sum() / len(y)
    return accuracy.item()

#### **Test NCF**

In [50]:
# use gpu 
use_gpu = 6
if use_gpu == -1:
    device = 'cpu'
else:
    device = torch.device('cuda:{}'.format(str(use_gpu)) if torch.cuda.is_available() else 'cpu')

# load users, items and ratings as tensors
users = torch.tensor(user_list, device = device)
items = torch.tensor(item_list, device = device)
ratings = torch.tensor(rating_list, device = device)
ratings = ratings.reshape((len(ratings), 1))

# define model and it's parameters
n_factors = 64
lr = 65
T = 501

# for lr in [1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 1000]:
for lr in range(60, 71):

    for T in [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550]:

        model = NCF(n_users, n_items, n_factors)
        # model = CollaborativeFiltering(n_users, n_items, n_factors)
        model.to(device)
        # optimizer = torch.optim.Adam(model.parameters())
        optimizer = torch.optim.SGD(model.parameters(), lr = lr)

        torch.manual_seed(0)
        for layer in model.children():
            layer.reset_parameters()

        loss_fn = nn.BCELoss(reduction = 'mean')
        model.train()

        for i in range(T):
            y_hat = model(users, items)
            loss = loss_fn(y_hat, ratings)
            if (i % (T - 1) == 0) and  (i > 0):
                print('lr: ', lr, ' T = ', i + 1, ' loss: ', loss.item(), ' acc: ', get_accuracy(y_hat, ratings))

            # use torch.optim optimizer to compute gradients
            optimizer.zero_grad()
            loss.backward(retain_graph=True, create_graph=False)
            optimizer.step()
        print('lr: ', lr, ' T: ', T, ' loss: ', loss.item(), ' acc: ', get_accuracy(y_hat, ratings))
    print()

# LR = 62 and T = 250 seems to do the job very well 

lr:  60  T =  50  loss:  18.998760223388672  acc:  0.729640007019043
lr:  60  T:  50  loss:  18.998760223388672  acc:  0.729640007019043
lr:  60  T =  100  loss:  19.90110969543457  acc:  0.7626699805259705
lr:  60  T:  100  loss:  19.90110969543457  acc:  0.7626699805259705
lr:  60  T =  150  loss:  11.323873519897461  acc:  0.8144049644470215
lr:  60  T:  150  loss:  11.323873519897461  acc:  0.8144049644470215
lr:  60  T =  200  loss:  14.165251731872559  acc:  0.7846999764442444
lr:  60  T:  200  loss:  14.165251731872559  acc:  0.7846999764442444
lr:  60  T =  250  loss:  6.241816520690918  acc:  0.8203099966049194
lr:  60  T:  250  loss:  6.241816520690918  acc:  0.8203099966049194
lr:  60  T =  300  loss:  13.87374496459961  acc:  0.7644099593162537
lr:  60  T:  300  loss:  13.87374496459961  acc:  0.7644099593162537
lr:  60  T =  350  loss:  3.9853928089141846  acc:  0.7546049952507019
lr:  60  T:  350  loss:  3.9853928089141846  acc:  0.7546049952507019
lr:  60  T =  400  loss